In [1]:
!pip install pickle5

     |████████████████████████████████| 133kB 13.1MB/s 
  Created wheel for pickle5: filename=pickle5-0.0.11-cp37-cp37m-linux_x86_64.whl size=219268 sha256=9849bba51d74554f8c5018966185b2691430ed58f6dc231a70bdf64e0e6b4cfc
  Stored in directory: /root/.cache/pip/wheels/a6/90/95/f889ca4aa8b0e0c7f21c8470b6f5d6032f0390a3a141a9a3bd
Successfully built pickle5


In [2]:
import pickle5
import numpy as np
from sklearn.model_selection import train_test_split

# import the necessary packages
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers.experimental import preprocessing

import matplotlib.pyplot as plt

In [3]:
with open('data.pickle', 'rb') as handle:
    data = pickle5.load(handle)

In [4]:
total_sample = len(data["joints_status"])
print(total_sample)

4096


In [5]:
a = [[1,1], [1, 2]]
b = [[1,1], [1, 2]]
np.hstack((a,b,a))

array([[1, 1, 1, 1, 1, 1],
       [1, 2, 1, 2, 1, 2]])

In [11]:
X = []
y = []
for elem in range(total_sample):
    X.append(data["ee_position"][elem])
    y.append(data["joints_status"][elem])

#X_pre = [[0.6966, -0.3300, 2.260]]
#y_pre = [[0, 0, 0, 0, 0, 0, 0]]
#for elem in range(total_sample):
#    X_pre.append(data["ee_position"][elem])
#    y_pre.append(data["joints_status"][elem])

#X = np.hstack((X,X_pre[:-1],y_pre[:-1]))
X = np.array(X)
y = np.array(y)

num_elems = X.shape[0]
print(num_elems)
print(X[567])
print(y[567])

4096
[ 1.08310259 -0.55177735  1.79666475  0.6966     -0.33        2.26
  0.          0.          0.          0.          0.          0.
  0.        ]
[-1.5707      0.52356667 -1.5707      1.5707     -0.52356667  1.5707
  0.        ]


In [ ]:
X[0]

array([0.27728424, 0.09114009, 1.43583368])

In [ ]:
X[0] + 2e-2

array([0.29728424, 0.11114009, 1.45583368])

In [15]:
index_to_delete = []
for i in range(num_elems):

  for j in range(num_elems):
    if (i != j) and np.linalg.norm(X[i]-X[j]) < 2e-2:
      if j not in index_to_delete:
        index_to_delete.append(j)
len(index_to_delete)

3302

In [16]:
X = np.delete(X, index_to_delete, 0)
y = np.delete(y, index_to_delete, 0)
X.shape[0]

794

In [ ]:
y_abs = abs(y)
u, indices = np.unique(a, return_index=True)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()
print(scaler_x.fit(X_train))
print(scaler_y.fit(y_train))

MinMaxScaler(copy=True, feature_range=(0, 1))
MinMaxScaler(copy=True, feature_range=(0, 1))


In [ ]:
def unique_verbose(x1, x2, y1, y2):
  print()
  print("equal")
  print(x1)
  print(y1)
  print(x2)
  print(y2)

duplicate_index = []
for i in range(X.shape[0]):
  for j in range(X.shape[0]):
    if j != i:
      if all(abs(X[i] - X[j]) < 1e-2):
        #unique_verbose(X[i], X[y], y[i], y[j])
        duplicate_index.append(i)
        break
len(duplicate_index)

2782

In [ ]:
X = np.delete(X, duplicate_index, 0)
y = np.delete(y, duplicate_index, 0)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=True, random_state=128)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.09, shuffle=True, random_state=128)

In [ ]:
normalizer = preprocessing.Normalization(input_shape=[3,])
normalizer.adapt(X_train)

In [ ]:
X_train.shape

(96, 3)

In [ ]:
X[0]

array([0.27728424, 0.09114009, 1.43583368])

In [ ]:
y[0]

array([-1.5707, -1.5707, -1.5707, -1.5707, -1.5707, -1.5707,  0.    ])

In [ ]:
import plotly.graph_objects as go
import numpy as np

fig = go.Figure(data=[go.Scatter3d(x=X[:,0], y=X[:,1], z=X[:,2],
                                   mode='markers',
                                      marker=dict(
                                        size=2,
                                        colorscale='Viridis',   # choose a colorscale
                                        opacity=0.8
                                    ))])
fig.show()

In [13]:
def create_model():
    model = Sequential()
    model.add(Dense(64, activation="relu"))
    model.add(Dense(7))
    sgd = tf.optimizers.SGD(0.001, momentum = 0.3, nesterov = True)
    adam = tf.optimizers.Adam(learning_rate=0.01)
    model.compile(loss="mean_squared_error", optimizer=adam)
    return model

In [23]:
# train the model using SGD
print("[INFO] training network...")
model = create_model()
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=1000, verbose = 2)

[INFO] training network...
Epoch 1/1000
21/21 - 0s - loss: 0.8063 - val_loss: 0.7935
Epoch 2/1000
21/21 - 0s - loss: 0.7488 - val_loss: 0.7787
Epoch 3/1000
21/21 - 0s - loss: 0.7315 - val_loss: 0.7893
Epoch 4/1000
21/21 - 0s - loss: 0.7264 - val_loss: 0.7686
Epoch 5/1000
21/21 - 0s - loss: 0.7282 - val_loss: 0.7879
Epoch 6/1000
21/21 - 0s - loss: 0.7206 - val_loss: 0.7702
Epoch 7/1000
21/21 - 0s - loss: 0.7162 - val_loss: 0.7535
Epoch 8/1000
21/21 - 0s - loss: 0.7155 - val_loss: 0.7682
Epoch 9/1000
21/21 - 0s - loss: 0.7102 - val_loss: 0.7672
Epoch 10/1000
21/21 - 0s - loss: 0.7054 - val_loss: 0.7613
Epoch 11/1000
21/21 - 0s - loss: 0.7059 - val_loss: 0.7616
Epoch 12/1000
21/21 - 0s - loss: 0.7067 - val_loss: 0.7700
Epoch 13/1000
21/21 - 0s - loss: 0.7018 - val_loss: 0.7656
Epoch 14/1000
21/21 - 0s - loss: 0.7048 - val_loss: 0.7645
Epoch 15/1000
21/21 - 0s - loss: 0.7051 - val_loss: 0.7531
Epoch 16/1000
21/21 - 0s - loss: 0.7000 - val_loss: 0.7702
Epoch 17/1000
21/21 - 0s - loss: 0.716

In [ ]:
model.evaluate(X_test, y_test)

3/3 [==============================] - 0s 5ms/step - loss: 0.7728


0.7727862596511841

In [ ]:
# Save the weights
model.save_weights('./checkpoints/nn_checkpoint_inverse_k')

In [ ]:
!zip -r filename.zip ./checkpoints

  adding: checkpoints/ (stored 0%)
  adding: checkpoints/checkpoint (deflated 50%)
  adding: checkpoints/nn_checkpoint_inverse_k.data-00000-of-00001 (deflated 15%)
  adding: checkpoints/nn_checkpoint_inverse_k.index (deflated 58%)


In [20]:
print(model.predict(X_val[0].reshape(1,3)))
print(y_test[0])

ValueError: ignored

In [22]:
print(model.predict(X_test[0].reshape(1,13)))
print(y_test[0])

[[-0.15862639  0.40959573 -0.05601184 -0.02949207  0.07645656 -0.028664
  -0.01326445]]
[-0.52356667 -1.5707     -0.52356667  0.52356667  0.52356667 -0.52356667
  0.        ]


In [ ]:
from sklearn.datasets import make_regression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
# Create the SVR regressor
svr = SVR(epsilon=0.2)

# Create the Multioutput Regressor
mor = MultiOutputRegressor(svr)

In [ ]:
# Train the regressor
mor = mor.fit(X_train, y_train)

# Generate predictions for testing data
y_pred = mor.predict(X_test)

# Evaluate the regressor
mse_one = mean_squared_error(y_test[0,:], y_pred[0,:])
mse_two = mean_squared_error(y_test[1,:], y_pred[1,:])
print(f'MSE for first regressor: {mse_one} - second regressor: {mse_two}')
mae_one = mean_absolute_error(y_test[0,:], y_pred[0,:])
mae_two = mean_absolute_error(y_test[1,:], y_pred[1,:])
print(f'MAE for first regressor: {mae_one} - second regressor: {mae_two}')

MSE for first regressor: 1.0513426941901076 - second regressor: 1.2701429165781557
MAE for first regressor: 0.749546365608773 - second regressor: 0.9704501156290278


In [ ]:
y_test[0,:]

array([ 0.52356667, -1.5707    ,  1.5707    ,  1.5707    ,  0.52356667,
        1.5707    ,  0.        ])

In [ ]:
y_pred[0,:]

array([ 0.46502645, -1.20168726, -0.12025611,  0.28743933,  0.31223449,
       -0.06302265,  0.        ])